In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint, solve_ivp

plt.rcParams.update({'font.size': 12})

---------------
# Epidemija
---------------

## Osnovni model epidemije

In [ ]:
# Constants
alpha = 0.8  # Alpha and beta values
beta = 0.2   # Alpha and beta values

def f(t, y):
    D, B, I = y  # Unpack the state variables

    # Compute the derivatives
    dDdt = -alpha * D * B
    dBdt = alpha * D * B - beta * B
    dIdt = beta * B

    return [dDdt, dBdt, dIdt]  # Return the derivatives as a list

# Initial values and desired time span
initial_D = 0.9
initial_B = 0.1
initial_I = 0.
y0 = [initial_D, initial_B, initial_I]  # Initial values

t_initial = 0.
t_final = 50.
num_points = 100
ts = np.linspace(t_initial, t_final, num_points)
t_span = (t_initial, t_final)  # Time span

# Solve the differential equations
solution = solve_ivp(f, t_span, y0, t_eval=ts, method='RK45')

# The solution is in solution.y
D_solution, B_solution, I_solution = solution.y

# Plot the solutions
plt.figure(figsize=(10,5))
plt.grid()
plt.title('Potek epidemije za ' + r'$\alpha=$' +f'{alpha} in ' + r'$\beta=$' + f'{beta}')
plt.ylabel('% Populacije')
plt.xlabel('t')
plt.plot(ts, D_solution, label='Dovzetni')
plt.plot(ts, B_solution, label='Bolni')
plt.plot(ts, I_solution, label='Imuni')
plt.legend()

- Prikaz velikosti vrhova in časa vrhova v odvisnosti od parametrov alpha in beta

In [ ]:
alphas = np.arange(0.1, 1.1, 0.1)
betas = np.arange(0.1, 1.1, 0.1)

Bmax1 = np.zeros((len(alphas), len(betas)))
tmax1 = np.zeros((len(alphas), len(betas)))

# Initial values and desired time span
initial_D = 0.9
initial_B = 0.1
initial_I = 0.

t_initial = 0.
t_final = 50.
num_points = 100
ts = np.linspace(t_initial, t_final, num_points)

# Initial conditions
y0 = [initial_D, initial_B, initial_I]  # Initial values

# Time span
t_span = (t_initial, t_final)  # Time span

for i, alpha in enumerate(alphas):

    for j, beta in enumerate(betas):

        def f(t, y):
            D, B, I = y  # Unpack the state variables

            # Compute the derivatives
            dDdt = -alpha * D * B
            dBdt = alpha * D * B - beta * B
            dIdt = beta * B

            return [dDdt, dBdt, dIdt]  # Return the derivatives as a list
        
        # Solve the differential equations
        solution = solve_ivp(f, t_span, y0, t_eval=ts, method='RK45')

        # The solution is in solution.y
        D_solution, B_solution, I_solution = solution.y

        Bmax1[i][j] = B_solution.max()
        tmax1[i][j] = ts[np.argmax(B_solution)]

In [ ]:
# Create a figure and axis for the heatmap
fig, ax = plt.subplots(figsize=(10,10))

# Create the heatmap of Bmax1
cax = ax.matshow(Bmax1, cmap='cividis')

# Set labels for the x and y axes
ax.set_xticks(np.arange(len(betas)))
ax.set_yticks(np.arange(len(alphas)))
ax.set_xticklabels([f'{beta:.1f}' for beta in betas])
ax.set_yticklabels([f'{alpha:.1f}' for alpha in alphas])

# Label the axes
plt.xlabel(r'$\beta$')
plt.ylabel(r'$\alpha$')

# Add a colorbar
cbar = fig.colorbar(cax, label=r'$B_{max}$')

# Set the title
plt.title(r'$B_{max}$ in $t(B_{max})$ v odvisnosti od $\alpha$ in $\beta$')

# Annotate tmax1 values on each square
for i in range(len(alphas)):
    for j in range(len(betas)):
        if j == 0 and i > 4:
            ax.text(j, i, f'{tmax1[i][j]:.2f}', ha='center', va='center', color='black')
        else:
            ax.text(j, i, f'{tmax1[i][j]:.2f}', ha='center', va='center', color='whitesmoke')

# Display the heatmap
plt.show()


- prikaz velikosti vrha in čas vrha v odvisnosti od I(0) in B(0)

In [ ]:
# Constants
alpha = 0.8  # Alpha and beta values
beta = 0.2   # Alpha and beta values

def f(t, y):
    D, B, I = y  # Unpack the state variables

    # Compute the derivatives
    dDdt = -alpha * D * B
    dBdt = alpha * D * B - beta * B
    dIdt = beta * B

    return [dDdt, dBdt, dIdt]  # Return the derivatives as a list

t_initial = 0.
t_final = 50.
num_points = 100
ts = np.linspace(t_initial, t_final, num_points)
t_span = (t_initial, t_final)  # Time span

initial_Bs = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2]

fig, ax = plt.subplots(figsize=(10,10))
ax.grid()
ax.set_title(r'Vrh epidemije v odvisnosti od začetnega števila precepljenih $I_0$')
ax.set_ylabel(r'$B_{max}$')
ax.set_xlabel(r'$I(0)$')

for initial_B in initial_Bs:
    initial_Is = np.arange(0, 1-initial_B+0.1, 0.02)
    Bmaxs = np.zeros_like(initial_Is)

    for i, initial_I in enumerate(initial_Is):
        initial_D = 1 - initial_B - initial_I

        y0 = [initial_D, initial_B, initial_I] # Initial values

        solution = solve_ivp(f, t_span, y0, t_eval=ts, method='RK45')

        # The solution is in solution.y
        D_solution, B_solution, I_solution = solution.y
        Bmaxs[i] = B_solution.max()

    ax.plot(initial_Is, Bmaxs, label=r'$B(0)=$'+f'{initial_B}')

ax.legend()

# Model epidemije z večimi stadiji okužbe

In [ ]:
# Constants
alpha = 0.8
beta = 0.2
alpha1 = 0.25 * alpha  # Alpha and beta values
alpha2 = alpha  # Alpha and beta values
alpha3 = 0.05 * alpha  # Alpha and beta values
beta1 = 3*beta  # Alpha and beta values
beta2 = 3*beta  # Alpha and beta values
beta3 = 3*beta  # Alpha and beta values

def f(t, y):
    D, B1, B2, B3, I = y  # Unpack the state variables

    # Compute the derivatives
    dDdt = -alpha1 * D * B1 - alpha2 * D * B2 - alpha3 * D * B3 
    dB1dt = alpha1 * D * B1 + alpha2 * D * B2 + alpha3 * D * B3 - beta1 * B1
    dB2dt = beta1 * B1 - beta2 * B2
    dB3dt = beta2 * B2 - beta3 * B3
    dIdt = beta3 * B3

    return [dDdt, dB1dt, dB2dt, dB3dt, dIdt]  # Return the derivatives as a list

# Initial values and desired time span
initial_D = 0.9
initial_B1 = 0.1
initial_B2 = 0.0
initial_B3 = 0.0
initial_I = 0.
y0 = [initial_D, initial_B1, initial_B2, initial_B3, initial_I]  # Initial values

t_initial = 0.
t_final = 50.
num_points = 100
ts = np.linspace(t_initial, t_final, num_points)
t_span = (t_initial, t_final)  # Time span

# Solve the differential equations
solution = solve_ivp(f, t_span, y0, t_eval=ts, method='RK45')

# The solution is in solution.y
D_solution, B1_solution, B2_solution, B3_solution, I_solution = solution.y

# Plot the solutions
plt.figure(figsize=(10,5))
plt.grid()
plt.title('Potek epidemije za ' + r'$\alpha=$' +f'{alpha} in ' + r'$\beta=$' + f'{beta}')
plt.ylabel('% Populacije')
plt.xlabel('t')
plt.plot(ts, D_solution, label='Dovzetni')
plt.plot(ts, B1_solution + B2_solution + B3_solution, label='Bolni')
plt.plot(ts, I_solution, label='Imuni')
plt.plot(ts, B1_solution, alpha=0.65, label='Bolni1')
plt.plot(ts, B2_solution, alpha=0.65, label='Bolni2')
plt.plot(ts, B3_solution, alpha=0.65, label='Bolni3')
plt.legend(loc='right')

- Prikaz velikosti vrhova in časa vrhova v odvisnosti od parametrov alpha in beta

In [ ]:
alphas = np.arange(0.1, 1.1, 0.1)
betas = np.arange(0.1, 1.1, 0.1)

Bmax2 = np.zeros((len(alphas), len(betas)))
tmax2 = np.zeros((len(alphas), len(betas)))

# Initial values and desired time span
initial_D = 0.9
initial_B1 = 0.1
initial_B2 = 0.0
initial_B3 = 0.0
initial_I = 0.
y0 = [initial_D, initial_B1, initial_B2, initial_B3, initial_I]  # Initial values

t_initial = 0.
t_final = 50.
num_points = 100
ts = np.linspace(t_initial, t_final, num_points)
t_span = (t_initial, t_final)  # Time span

for i, alpha in enumerate(alphas):

    alpha1 = 0.25 * alpha  # Alpha and beta values
    alpha2 = alpha  # Alpha and beta values
    alpha3 = 0.05 * alpha  # Alpha and beta values

    for j, beta in enumerate(betas):
        beta1 = 3*beta  # Alpha and beta values
        beta2 = 3*beta  # Alpha and beta values
        beta3 = 3*beta  # Alpha and beta values

        def f(t, y):
            D, B1, B2, B3, I = y  # Unpack the state variables
        
            # Compute the derivatives
            dDdt = -alpha1 * D * B1 - alpha2 * D * B2 - alpha3 * D * B3 
            dB1dt = alpha1 * D * B1 + alpha2 * D * B2 + alpha3 * D * B3 - beta1 * B1
            dB2dt = beta1 * B1 - beta2 * B2
            dB3dt = beta2 * B2 - beta3 * B3
            dIdt = beta3 * B3
        
            return [dDdt, dB1dt, dB2dt, dB3dt, dIdt]  # Return the derivatives as a list
        
        # Solve the differential equations
        solution = solve_ivp(f, t_span, y0, t_eval=ts, method='RK45')

        # The solution is in solution.y
        D_solution, B1_solution, B2_solution, B3_solution, I_solution = solution.y

        Bmax2[i][j] = (B1_solution + B2_solution + B3_solution).max()
        tmax2[i][j] = ts[np.argmax((B1_solution + B2_solution + B3_solution))]

In [ ]:
# Create a figure and axis for the heatmap
fig, ax = plt.subplots(figsize=(12,12))

# Create the heatmap of Bmax2
cax = ax.matshow(Bmax2, cmap='cividis')

# Set labels for the x and y axes
ax.set_xticks(np.arange(len(betas)))
ax.set_yticks(np.arange(len(alphas)))
ax.set_xticklabels([f'{beta:.1f}' for beta in betas])
ax.set_yticklabels([f'{alpha:.1f}' for alpha in alphas])

# Label the axes
plt.xlabel('Beta')
plt.ylabel('Alpha')

# Add a colorbar
cbar = fig.colorbar(cax, label=r'$B_{max}$')

# Set the title
plt.title(r'$B_{max}$ in $t(B_{max})$ v odvisnosti od $\alpha$ in $\beta$')

# Annotate tmax2 values on each square
for i in range(len(alphas)):
    for j in range(len(betas)):
        if j == 0 and i > 4:
            ax.text(j, i, f'{tmax2[i][j]:.2f} s', ha='center', va='center', color='black')
        else:
            ax.text(j, i, f'{tmax2[i][j]:.2f} s', ha='center', va='center', color='whitesmoke')

# Display the heatmap
plt.show()


- prikaz velikosti vrha in čas vrha v odvisnosti od I(0) in B(0)

In [ ]:
# Constants
alpha = 0.8  # Alpha and beta values
beta = 0.2   # Alpha and beta values
alpha1 = 0.25 * alpha  # Alpha and beta values
alpha2 = alpha  # Alpha and beta values
alpha3 = 0.05 * alpha  # Alpha and beta values
beta1 = 3*beta  # Alpha and beta values
beta2 = 3*beta  # Alpha and beta values
beta3 = 3*beta  # Alpha and beta values

def f(t, y):
    D, B1, B2, B3, I = y  # Unpack the state variables
        
    # Compute the derivatives
    dDdt = -alpha1 * D * B1 - alpha2 * D * B2 - alpha3 * D * B3 
    dB1dt = alpha1 * D * B1 + alpha2 * D * B2 + alpha3 * D * B3 - beta1 * B1
    dB2dt = beta1 * B1 - beta2 * B2
    dB3dt = beta2 * B2 - beta3 * B3
    dIdt = beta3 * B3
        
    return [dDdt, dB1dt, dB2dt, dB3dt, dIdt]  # Return the derivatives as a list
        

t_initial = 0.
t_final = 50.
num_points = 100
ts = np.linspace(t_initial, t_final, num_points)
t_span = (t_initial, t_final)  # Time span

initial_B1s = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2]

fig, ax = plt.subplots(figsize=(12,12))
ax.grid()
ax.set_title(r'Vrh epidemije v odvisnosti od začetnega števila precepljenih $I_0$')
ax.set_ylabel(r'$B_{max}$')
ax.set_xlabel(r'$I(0)$')

for initial_B1 in initial_B1s:
    initial_Is = np.arange(0, 1-initial_B1+0.1, 0.02)
    Bmaxs = np.zeros_like(initial_Is)

    for i, initial_I in enumerate(initial_Is):
        initial_D = 1 - initial_B1 - initial_I

        y0 = [initial_D, initial_B1, 0, 0, initial_I] # Initial values

        solution = solve_ivp(f, t_span, y0, t_eval=ts, method='RK45')

        # The solution is in solution.y
        D_solution, B1_solution, B2_solution, B3_solution, I_solution = solution.y
        Bmaxs[i] = (B1_solution + B2_solution + B3_solution).max()

    ax.plot(initial_Is, Bmaxs, label=r'$B1(0)=$'+f'{initial_B1}')

ax.legend()

### primerjava modelov

In [ ]:
# Create a figure and axis for the heatmap
fig, ax = plt.subplots(figsize=(12,12))

# Create the heatmap of Bmax2 - Bmax1
cax = ax.matshow(Bmax2 - Bmax1, cmap='cividis')

# Set labels for the x and y axes
ax.set_xticks(np.arange(len(betas)))
ax.set_yticks(np.arange(len(alphas)))
ax.set_xticklabels([f'{beta:.1f}' for beta in betas])
ax.set_yticklabels([f'{alpha:.1f}' for alpha in alphas])

# Label the axes
plt.xlabel('Beta')
plt.ylabel('Alpha')

# Add a colorbar
cbar = fig.colorbar(cax, label=r'$B_{max}$')

# Set the title
plt.title(r'$B_{max}^{(Stadiji)} - B_{max}^{(Zač)}$ in $t(B_{max}^{(Stadiji)}) - t(B_{max}^{(Zač)})$ v odvisnosti od $\alpha$ in $\beta$')

# Annotate tmax2 values on each square
for i in range(len(alphas)):
    for j in range(len(betas)):
        if j > i-1:
            ax.text(j, i, f'{tmax2[i][j]-tmax1[i][j]:.2f} s', ha='center', va='center', color='black')
        elif j > i-2 and j > 0:
            ax.text(j, i, f'{tmax2[i][j]-tmax1[i][j]:.2f} s', ha='center', va='center', color='black')
        elif j > i-3 and j > 3:
            ax.text(j, i, f'{tmax2[i][j]-tmax1[i][j]:.2f} s', ha='center', va='center', color='black')
        else:
            ax.text(j, i, f'{tmax2[i][j]-tmax1[i][j]:.2f} s', ha='center', va='center', color='whitesmoke')

# Display the heatmap
plt.show()


In [ ]:
# Constants
alpha = 0.8  # Alpha and beta values
beta = 0.2   # Alpha and beta values
alpha1 = 0.25 * alpha  # Alpha and beta values
alpha2 = alpha  # Alpha and beta values
alpha3 = 0.05 * alpha  # Alpha and beta values
beta1 = 3*beta  # Alpha and beta values
beta2 = 3*beta  # Alpha and beta values
beta3 = 3*beta  # Alpha and beta values

def f2(t, y):
    D, B1, B2, B3, I = y  # Unpack the state variables
        
    # Compute the derivatives
    dDdt = -alpha1 * D * B1 - alpha2 * D * B2 - alpha3 * D * B3 
    dB1dt = alpha1 * D * B1 + alpha2 * D * B2 + alpha3 * D * B3 - beta1 * B1
    dB2dt = beta1 * B1 - beta2 * B2
    dB3dt = beta2 * B2 - beta3 * B3
    dIdt = beta3 * B3
        
    return [dDdt, dB1dt, dB2dt, dB3dt, dIdt]  # Return the derivatives as a list

def f1(t, y):
    D, B, I = y  # Unpack the state variables

    # Compute the derivatives
    dDdt = -alpha * D * B
    dBdt = alpha * D * B - beta * B
    dIdt = beta * B

    return [dDdt, dBdt, dIdt]  # Return the derivatives as a list

t_initial = 0.
t_final = 50.
num_points = 1000
ts = np.linspace(t_initial, t_final, num_points)
t_span = (t_initial, t_final)  # Time span

initial_Bs = np.linspace(0.025, 0.2, 60)# [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2]
Icrit1 = np.zeros_like(initial_Bs)
Icrit2 = np.zeros_like(initial_Bs)

for j, initial_B in enumerate(initial_Bs):
    initial_Is = np.arange(0, 1-initial_B+0.1, 0.002)
    Bmaxs1 = np.zeros_like(initial_Is)
    Bmaxs2 = np.zeros_like(initial_Is)

    for i, initial_I in enumerate(initial_Is):
        initial_D = 1 - initial_B - initial_I

        y0 = [initial_D, initial_B, initial_I] # Initial values
        solution1 = solve_ivp(f1, t_span, y0, t_eval=ts, method='RK45')
        D_solution_1, B_solution_1, I_solution_1 = solution1.y

        y0 = [initial_D, initial_B, 0, 0, initial_I] # Initial values
        solution2 = solve_ivp(f2, t_span, y0, t_eval=ts, method='RK45')
        D_solution_2, B1_solution_2, B2_solution_2, B3_solution_2, I_solution_2 = solution2.y
        B_solution_2 = B1_solution_2 + B2_solution_2 + B3_solution_2

        # The solution is in solution.y
        Bmaxs1[i] = B_solution_1.max()
        Bmaxs2[i] = B_solution_2.max()
    
    index1 = np.where(Bmaxs1 <= 0.2)[0][0]
    index2 = np.where(Bmaxs2 <= 0.2)[0][0]

    Icrit1[j] = initial_Is[index1]
    Icrit2[j] = initial_Is[index2]

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.grid()
ax.set_title(r'Kritična precepljenost $I_{crit}$ v odvisnosti od začetnega števila bolnih $B(0)$, $B_1(0)$')
ax.set_ylabel(r'$I_{crit}$')
ax.set_xlabel(r'$B(0)$, $B_1(0)$')
ax.scatter(initial_Bs, Icrit1, label='Začetni model')
ax.scatter(initial_Bs, Icrit2, label='Model s tremi stadiji')

ax.legend()